# Spark Connect in SparkAgent
This notebook shows how to use SparkAgent with Spark Connect. (go to [Spark Dataframe Agent](spark.ipynb) for basics.)

**NOTE: this agent calls the Python agent under the hood, which executes LLM generated Python code - this can be bad if the LLM generated Python code is harmful. Use cautiously.**